# Reimportando as bibliotecas

In [1]:
!pip install apache_beam
import re
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io.textio import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
pipeline_options = PipelineOptions(argv = None)
pipeline = beam.Pipeline(options = pipeline_options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 25.7 MB/s 
     |████████████████████████████████| 2.4 MB 50.8 MB/s 
     |████████████████████████████████| 151 kB 52.3 MB/s 
     |████████████████████████████████| 270 kB 57.2 MB/s 
     |████████████████████████████████| 508 kB 58.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 47 kB 5.5 MB/s 
     |████████████████████████████████| 1.0 MB 57.2 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=78acd96c5e6b121df23cc66d5f3d883bbe8744ecc08371a1693b20f807fb49f1
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c379e2bd5b023c411b329899fc3620e8324e1638acad7552d79ff1931ca7c38c
  Stored in directory: /root/.c

# Reimportando os métodos

In [2]:
def TextoParaLista(elemento,delimitador = '|'):
  """
  Recebe um texto e um limitador
  Retorna uma lista de elementos pelo delimitador
  """
  return elemento.split(delimitador)



In [3]:
colunas_dengue = [ #todas as colunas que vem do dataset
'id',
'data_iniSE',
'casos',
'ibge_code',
'cidade',
'uf',
'cep',
'latitude',
'longitude']

In [4]:
def ListaParaDicionario(elemento, colunas):
  """
  Recebe duas listas e retorna um dicionário
  """
  return dict(zip(colunas, elemento))

In [5]:
def TrataDatas(elemento):
  """
  Recebe um dicionário e cria um novo campo com ano-mês
  """
  elemento['ano-mes'] = '-'.join(elemento['data_iniSE'].split('-')[:2])
  #Supondo que temos a data 2022-05-18 o split retornará uma lista com ['2022','05','18']
  #Então fariaremos a lista usando um [:2] no final pegando as informações até o segundo elemento
  #Aplicamos também o join com o parâmetro '-' que nos retornará uma STRING no formato 2022-05
  return elemento

In [6]:
def ChaveUf(elemento):
  """
  Receber um dicionário
  Retornar uma tupla com o estado(uf) e o elemento
  """
  chave = elemento['uf']
  return (chave, elemento)

In [7]:
def CasosDengue(elemento):
  """
  Recebe uma tupla (RS, [{},{}])
  Retorna uma tupla ('RS-2014-12, 8')
  """
  uf, registros = elemento
  for registro in registros:
    if bool(re.search(r'\d', registro['casos'])):
      yield (f"{uf}-{registro['ano-mes']}", float(registro['casos'])) #Diferente do return o yield vai executar até retornar todos os elementos
    else:
      yield (f"{uf}-{registro['ano-mes']}", 0.0)

In [8]:
def ChaveUfMes(elemento):
  """
  Receber uma lista de elementos
  Retorna uma tupla ('UF-ANO-MES', casos)
  """
  data, mm, uf = elemento
  ano_mes = '-'.join(data.split('-')[:2])
  chave = f'{uf}-{ano_mes}'
  if float(mm) < 0:
    mm = 0.0
  else:
    mm = float(mm)
  return chave, mm

In [9]:
def arredonda(elemento):
  """
  Recebe um elemento 
  retorna o float com uma ou duas casas decimais.
  """
  chave, mm = elemento
  return (chave, round(mm,1))

In [10]:
def FiltraCamposVazios(elemento):
  """
  Remove elementos que contenam chaves vazias
  """
  chave, dados = elemento
  if all([dados['chuvas'], dados['dengue']]):
      return True
  return False

In [11]:
def DescompactarElementos(elem):
  """
  Recebe uma tupla ('CE-2015-01', {'chuvas': [85.8], 'dengue': [175.0]})
  retorna uma tupla ('CE', '2015', '11', '0.4', '21.0')
  """
  chave, dados = elem
  chuva = dados['chuvas'][0]
  dengue = dados['dengue'][0]
  uf, ano, mes = chave.split('-')
  return uf, ano, mes, str(chuva), str(dengue)

In [12]:
def PrepararCSV(elem, delimitador = ';'):
  """
  Recebe uma tupla ('CE', 2015, 01, 85.8, 175.0)
  Retornar uma string delimitada "CE;2015;01;85.8;175.0"
  """
  return f"{delimitador}".join(elem)

# Pcollections

In [13]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) 
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Criar chave pelo estado" >> beam.Map(ChaveUf)
    | "Agrupor por estado" >> beam.GroupByKey()
    | "Descompactar casos de dengue" >> beam.FlatMap(CasosDengue)
    | "Soma dos casos pela chave" >> beam.CombinePerKey(sum) 

)

chuvas = (
    pipeline
    | "Leitura do dataset de chuvas" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/chuvas.csv', skip_header_lines=1)
    | "De texto para lista (chuvas)"  >> beam.Map(TextoParaLista,delimitador = ',') 
    | "Criando chave UF-ANO-MES" >> beam.Map(ChaveUfMes)
    | "Combinando o total de chuvas pela chave" >> beam.CombinePerKey(sum)
    | "Arredondar resultados de chuvas" >> beam.Map(arredonda)
)

resultado = (
    
    ({'chuvas': chuvas,'dengue': dengue})
    | "Mesclar pcols" >> beam.CoGroupByKey()
    | "Filtrar dados vazios" >> beam.Filter(FiltraCamposVazios)
    | "Descompactar elementos" >> beam.Map(DescompactarElementos)
    | "Preparar csv" >> beam.Map(PrepararCSV)
)

header = 'UF;ANO;MES;CHUVA;DENGUE'

resultado | 'Criar arquivo CSV' >> WriteToText('resultado', file_name_suffix='.csv', header = header)

pipeline.run()


# Analisando os dados

O Processamento anterior dura em torno de 5 a 10 minutos, pois analisará os dois datasets. Agora, iremos mostrar esses dados. 

In [14]:
import pandas as pd

In [15]:
arquivo = '/content/resultado-00000-of-00001.csv'
df = pd.read_csv(arquivo, delimiter = ';')

In [17]:
df.head() #Mostrando os 5 primeiros resultados

,UF,ANO,MES,CHUVA,DENGUE
0,SP,2015,1,4465.0,772.0
1,SP,2015,2,6595.0,3086.0
2,SP,2015,3,6210.6,8824.0
3,SP,2015,4,1862.0,9932.0
4,SP,2015,5,2756.6,8919.0


In [18]:
df.shape #420 linhas e 5 colunas

(420, 5)

In [22]:
df.groupby(['UF', 'ANO'])[['CHUVA','DENGUE']].mean().reset_index()

,UF,ANO,CHUVA,DENGUE
0,CE,2015,771.783333,5014.250000
1,CE,2016,533.700000,5433.083333
2,CE,2017,775.350000,5519.666667
3,CE,2018,797.950000,1070.083333
4,CE,2019,809.000000,2559.250000
5,ES,2015,575.650000,3734.000000
6,ES,2016,733.916667,4560.750000
7,ES,2017,903.883333,1028.666667
8,ES,2018,1399.166667,1407.500000
9,ES,2019,1122.350000,6716.083333
